# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pywin32 302 (c:\users\yuenche\anaconda3\envs\mlops-workshop-local\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pywin32 302 (c:\users\yuenche\anaconda3\envs\mlops-workshop-local\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pywin32 302 (c:\users\yuenche\anaconda3\envs\mlops-workshop-local\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).
Failure while loading azureml_run_type_providers. Fai

## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [2]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmp752ibsmy\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet
[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmprgoewebz\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=2\part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet
[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmpcnba_s_2\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=3\part-00196-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2766-1.c000.snappy.parquet
[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmp5mvy3xf6\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=4\part-00121-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2691-1.c000.snappy.parquet
[Info] read from C:\User

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
473916,2,2016-01-08 20:21:25,2016-01-08 21:04:29,1,4.72,None,None,-73.860535,40.832729,-73.864464,...,2,30.5,0.5,0.5,0.3,0.00,0.0,NaN,31.80,1.0
1139475,2,2016-01-08 23:53:19,2016-01-09 00:02:12,1,1.25,None,None,-73.953690,40.803562,-73.949219,...,2,7.5,0.5,0.5,0.3,0.00,0.0,NaN,8.80,1.0
1161987,2,2016-01-10 07:28:34,2016-01-10 07:44:35,1,2.67,None,None,-73.912880,40.756077,-73.923035,...,2,11.5,0.0,0.5,0.3,0.00,0.0,NaN,12.30,1.0
59171,1,2016-01-06 00:05:12,2016-01-06 00:13:51,1,1.30,None,None,-73.977493,40.689770,-73.997520,...,1,8.0,0.5,0.5,0.3,1.85,0.0,NaN,11.15,1.0
1165442,2,2016-01-10 10:38:59,2016-01-10 10:58:03,2,7.16,None,None,-73.947815,40.825157,-74.001411,...,1,23.0,0.0,0.5,0.3,5.95,0.0,NaN,29.75,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506360,2,2016-12-20 19:16:08,2016-12-20 19:29:42,6,2.66,41,236,NaN,NaN,NaN,...,2,11.5,1.0,0.5,0.3,0.00,0.0,NaN,13.30,1.0
398689,1,2016-12-17 00:15:49,2016-12-17 00:28:07,3,3.70,116,243,NaN,NaN,NaN,...,1,14.0,0.5,0.5,0.3,3.05,0.0,NaN,18.35,1.0
1181960,2,2016-12-12 20:58:34,2016-12-12 21:06:29,1,1.85,61,188,NaN,NaN,NaN,...,2,8.0,0.5,0.5,0.3,0.00,0.0,NaN,9.30,1.0
350233,2,2016-12-16 06:33:36,2016-12-16 06:48:00,1,3.49,130,205,NaN,NaN,NaN,...,2,14.0,0.0,0.5,0.3,0.00,0.0,NaN,14.80,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,tripType,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos
473916,2,2016-01-08 20:21:25,2016-01-08 21:04:29,1,4.72,None,None,-73.860535,40.832729,-73.864464,...,1.0,1,8,4,20,US,-0.866025,5.000000e-01,-0.433884,-0.900969
1139475,2,2016-01-08 23:53:19,2016-01-09 00:02:12,1,1.25,None,None,-73.953690,40.803562,-73.949219,...,1.0,1,8,4,23,US,-0.258819,9.659258e-01,-0.433884,-0.900969
1161987,2,2016-01-10 07:28:34,2016-01-10 07:44:35,1,2.67,None,None,-73.912880,40.756077,-73.923035,...,1.0,1,10,6,7,US,0.965926,-2.588190e-01,-0.781831,0.623490
59171,1,2016-01-06 00:05:12,2016-01-06 00:13:51,1,1.30,None,None,-73.977493,40.689770,-73.997520,...,1.0,1,6,2,0,US,0.000000,1.000000e+00,0.974928,-0.222521
1165442,2,2016-01-10 10:38:59,2016-01-10 10:58:03,2,7.16,None,None,-73.947815,40.825157,-74.001411,...,1.0,1,10,6,10,US,0.500000,-8.660254e-01,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506360,2,2016-12-20 19:16:08,2016-12-20 19:29:42,6,2.66,41,236,NaN,NaN,NaN,...,1.0,12,20,1,19,US,-0.965926,2.588190e-01,0.781831,0.623490
398689,1,2016-12-17 00:15:49,2016-12-17 00:28:07,3,3.70,116,243,NaN,NaN,NaN,...,1.0,12,17,5,0,US,0.000000,1.000000e+00,-0.974928,-0.222521
1181960,2,2016-12-12 20:58:34,2016-12-12 21:06:29,1,1.85,61,188,NaN,NaN,NaN,...,1.0,12,12,0,20,US,-0.866025,5.000000e-01,0.000000,1.000000
350233,2,2016-12-16 06:33:36,2016-12-16 06:48:00,1,3.49,130,205,NaN,NaN,NaN,...,1.0,12,16,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969


Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [6]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime
473916,2,2016-01-08 20:21:25,1,4.72,-73.860535,40.832729,-73.864464,40.837669,31.80,1,8,4,20,US,-0.866025,0.500000,-0.433884,-0.900969,2016-01-08
1139475,2,2016-01-08 23:53:19,1,1.25,-73.953690,40.803562,-73.949219,40.793907,8.80,1,8,4,23,US,-0.258819,0.965926,-0.433884,-0.900969,2016-01-08
1161987,2,2016-01-10 07:28:34,1,2.67,-73.912880,40.756077,-73.923035,40.730583,12.30,1,10,6,7,US,0.965926,-0.258819,-0.781831,0.623490,2016-01-10
59171,1,2016-01-06 00:05:12,1,1.30,-73.977493,40.689770,-73.997520,40.687969,11.15,1,6,2,0,US,0.000000,1.000000,0.974928,-0.222521,2016-01-06
1165442,2,2016-01-10 10:38:59,2,7.16,-73.947815,40.825157,-74.001411,40.748245,29.75,1,10,6,10,US,0.500000,-0.866025,-0.781831,0.623490,2016-01-10


### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [7]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmpxvls_2ou\https%3A\%2Fazureopendatastorage.azurefd.net\holidaydatacontainer\Processed\part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
19375,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],None,AR,2008-01-01
19376,Australia,New Year's Day,New Year's Day,None,AU,2008-01-01
19377,Austria,Neujahr,Neujahr,None,AT,2008-01-01
19378,Belarus,Новый год,Новый год,None,BY,2008-01-01
19379,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,None,BE,2008-01-01


Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [8]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,...,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime,normalizeHolidayName,isPaidTimeOff
6,2,2016-01-01 01:15:19,1,0.85,-73.961296,40.801838,-73.950127,40.806721,6.80,1,...,4,1,US,0.258819,9.659258e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
26,1,2016-01-01 02:06:22,1,0.00,-73.928703,40.860306,-73.929131,40.860168,3.80,1,...,4,2,US,0.500000,8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
47,2,2016-01-01 06:18:34,1,4.89,-73.946053,40.789253,-73.936661,40.849129,15.80,1,...,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969,2016-01-01,New Year's Day,True
57,1,2016-01-01 11:23:19,1,2.00,-73.978363,40.670513,-73.988129,40.690216,12.35,1,...,4,11,US,0.258819,-9.659258e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
99,2,2016-01-01 01:31:31,5,3.37,-73.958717,40.678566,-73.960861,40.718891,15.30,1,...,4,1,US,0.258819,9.659258e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23909,1,2016-12-25 00:35:32,1,6.60,NaN,NaN,NaN,NaN,34.30,12,...,6,0,US,0.000000,1.000000e+00,-0.781831,0.623490,2016-12-25,Christmas Day,True
23930,1,2016-12-25 22:29:01,2,3.60,NaN,NaN,NaN,NaN,21.00,12,...,6,22,US,-0.500000,8.660254e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23933,1,2016-12-25 09:10:16,1,1.20,NaN,NaN,NaN,NaN,6.30,12,...,6,9,US,0.707107,-7.071068e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23961,1,2016-12-25 13:46:25,1,0.90,NaN,NaN,NaN,NaN,8.80,12,...,6,13,US,-0.258819,-9.659258e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True


### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [9]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmpt69uriux\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmpt69uriux\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmpt69uriux\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-1.c000.snappy.parquet
[Info] read from C:\Users\yuenche\AppData\Local\Temp\tmpt69uriux\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00003-tid-738723883827836859-85e0759b-51fa-4430-8

In [10]:
weather_df

,longitude,datetime,latitude,wban,temperature,precipTime,precipDepth,usaf
204647,-73.867,2016-01-02 03:00:00,40.783,14732,2.8,NaN,NaN,725030
204670,-73.880,2016-01-22 13:51:00,40.779,14732,-4.4,1.0,0.0,725030
204694,-73.880,2016-01-08 02:51:00,40.779,14732,5.0,1.0,0.0,725030
204701,-73.880,2016-01-04 15:51:00,40.779,14732,-1.1,1.0,0.0,725030
204715,-73.880,2016-01-01 21:51:00,40.779,14732,4.4,1.0,0.0,725030
...,...,...,...,...,...,...,...,...
1248471,-73.967,2016-12-23 13:51:00,40.789,94728,4.4,1.0,0.0,725053
1248555,-73.967,2016-12-12 13:51:00,40.789,94728,5.0,1.0,0.0,725053
1248580,-73.967,2016-12-18 07:01:00,40.789,94728,3.9,NaN,NaN,725053
1248597,-73.967,2016-12-25 00:51:00,40.789,94728,7.8,1.0,0.0,725053


Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [11]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

,precipTime,temperature,precipDepth
datetime,,,
2016-01-01,1.0,5.197345,0.0
2016-01-02,1.0,2.567857,0.0
2016-01-03,1.0,3.846429,0.0
2016-01-04,1.0,0.123894,0.0
2016-01-05,6.0,-7.206250,0.0
2016-01-06,6.0,-0.896396,0.0
2016-01-07,6.0,3.180645,0.0
2016-01-08,1.0,4.384091,0.0
2016-01-09,6.0,6.710274,3.0


Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [12]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,24000.000000,24000.000000,24000.000000,12000.000000,12000.000000,12000.000000,12000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.794208,1.345333,2.808486,-73.835274,40.688824,-73.852433,40.697457,14.691013,6.500000,15.186458,3.217333,13.686125,-0.243148,-2.180299e-02,-0.067296,-0.059684,13.250958,13.911761,1062.618958
std,0.404287,1.007532,2.940811,2.806820,1.534274,2.552528,1.392813,12.020174,3.452124,8.491645,1.949040,6.645276,0.667355,7.036194e-01,0.714717,0.693636,10.338847,9.500252,2819.484263
min,1.000000,0.000000,0.000000,-115.171608,0.000000,-115.175011,0.000000,-140.300000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.010000,-73.961044,40.693691,-73.968065,40.695097,7.800000,3.750000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,6.591071,0.000000
50%,2.000000,1.000000,1.820000,-73.945759,40.746124,-73.945591,40.746500,11.300000,6.500000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,13.125893,10.000000
75%,2.000000,1.000000,3.500000,-73.918648,40.802212,-73.911858,40.790530,17.760000,9.250000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,22.944737,135.000000
max,2.000000,7.000000,61.800000,0.000000,40.948635,0.000000,41.344292,499.000000,12.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,31.303665,9999.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [13]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [14]:
final_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000,1.177400e+04,11774.000000,11774.000000,11774.000000,11774.000000,11774.000000
mean,1.790046,1.337269,2.867167,-73.937125,40.746424,-73.916934,40.734499,14.782607,3.498641,15.056565,3.184984,13.643027,-0.233467,-1.920391e-02,-0.063137,-0.049232,11.793188,10.297688,197.228639
std,0.407293,0.989528,2.961150,0.041526,0.056698,1.181135,0.652886,11.675174,1.708312,8.466282,1.962468,6.666910,0.671362,7.031915e-01,0.714144,0.695465,10.098047,8.577065,1245.662665
min,1.000000,1.000000,0.010000,-74.083778,40.572899,-74.374413,0.000000,0.050000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.060000,-73.961182,40.693735,-73.968588,40.695100,8.000000,2.000000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,3.504580,0.000000
50%,2.000000,1.000000,1.880000,-73.946434,40.745964,-73.946148,40.746231,11.300000,3.000000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,10.168182,3.000000
75%,2.000000,1.000000,3.580000,-73.919113,40.801838,-73.912487,40.789360,17.800000,5.000000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,17.239744,41.000000
max,2.000000,6.000000,54.800000,-73.741096,40.879993,0.000000,41.344292,401.800000,6.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,26.524107,9999.000000


## Train a model

The data is ready to train a machine learning model.

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [16]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    model=Ridge(alpha=100)
  elif algo_name == 'random_forest':
    model = RandomForestRegressor()
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [17]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [18]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression", 'random_forest']:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.863049198659428
MAPE: 0.16451002511624951
RMSE: 4.015577678259132

random_forest
R2: 0.8069889797090944
MAPE: 0.1618502578965397
RMSE: 4.767127809466536

